Basic Walkthrough:
     1) Define everything
     2) Create master bias file, Save master bias file  
     3) Open all other files, sub master bias, save  (*c?.b.fits)
     4) Remove cosmics from all file types except bias  (*c?.bc.fits)
     5) Open flats and create master skyflat file, save
     6) Open all remainging types and divide out master flat, then save  (*c?.bcf.fits)
     7) Open all remaining types and stitch together, save  (*full.bcf.fits)
     8) Use fibermap files to determine aperatures
     9) Use aperatures to cut out same regions in thar,comp,science
     10) Save all 256 to files with their header tagged name in filename, along with fiber num
     11) Assume no curvature within tiny aperature region; fit profile to 2d spec and sum to 1d
     12) Fit the lines in comp spectra, save file and fit solution
     13) Try to fit lines in thar spectra, save file and fit solution
     14) Apply to science spectra

In [1]:
import os
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Define input file numbers and other required information

Ex:

    Bias 597-626
    ThAr 627,635
    NeHgArXe 628,629,636,637
    Science 631-634
    Fibermaps 573-577


In [2]:
biass = np.arange(597,626+1).astype(int)
thar_lamps = np.asarray([627,635])
comp_lamps = np.asarray([628,629,636,637])
twiflats = np.arange(582,591+1).astype(int)
sciences = np.arange(631,634+1).astype(int)
fibermaps = np.arange(573,577+1).astype(int)

In [3]:
instrument = 'M2FS'
mask_name = 'A02'
cal_lamp = ['Xenon', 'Argon', 'HgNe']  # 'Xenon','Argon','Neon', 'HgNe'
cameras = ['r']
opamps = [1,2,3,4]

In [4]:
path_to_masks = os.path.abspath('../../OneDrive/Research/M2FSReductions')
mask_subdir = mask_name
raw_data_subdir =  'raw_data'
raw_filename_template = '{cam}{filenum:04d}c{opamp}.fits'

In [5]:
make_debug_plots = False
print_headers = True
cut_bias_cols = True

In [6]:
do_bias = False
do_cr   = False
do_flat = True
do_stitch = True
do_apcut = True
do_2dto1d = True
do_wavecalib = True
do_combine = True
do_zfit = True

###         Beginning of Code

In [7]:
date = np.datetime_as_string(np.datetime64('today', 'D'))

In [8]:
mask_dir = os.path.join(path_to_masks,mask_subdir)

In [9]:
raw_data_dir =    os.path.join(mask_dir, raw_data_subdir)
product_dir =     os.path.join(mask_dir,'data_products')
stitched_dir =    os.path.join(mask_dir,'stitched')
twod_dir =        os.path.join(mask_dir,'twods')
oned_dir =        os.path.join(mask_dir,'oneds')
calibrated_dir =  os.path.join(mask_dir,'calibrated_oned')
summedspec_dir =  os.path.join(mask_dir,'final_oned')
zfit_dir =        os.path.join(mask_dir,'zfits')

for folder in [ raw_data_dir, product_dir, stitched_dir, twod_dir, 
    oned_dir, calibrated_dir, summedspec_dir, zfit_dir]:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [10]:
fibermap_array_dict, fibermap_header_dict = None,None
twiflat_array_dict, twiflat_header_dict = None,None
bias_array_dict, bias_header_dict = None,None

In [11]:
headers = {}
data = {}
master_aux_data = {}
master_aux_headers = {}

In [12]:
filenumbers = {'bias':biass, 'thar':thar_lamps, 'comp': comp_lamps,
               'twiflat':twiflats, 'science': sciences, 'fibmap': fibermaps}

In [13]:
base_file_template =          '{cam}_{imtype}_{filenum:04d}_{maskname}_'
valadded_filename_template =  base_file_template+'c{opamp}{tags}.fits'
stitched_filename_template =  base_file_template+'stitched{tags}.fits'
twod_filename_template =      base_file_template+'{fibername}_2d{tags}.fits'
oned_filename_template =      base_file_template+'{fibername}_1d{tags}.fits'
combined_filename_template =  base_file_template+'{fibername}_1d{tags}.fits'

master_valadded_fname_template =  '{cam}_master{imtype}_{maskname}_c{opamp}{tags}.fits'
master_stitched_fname_template =  '{cam}_master{imtype}_{maskname}_stitched{tags}.fits'

In [14]:
common_func_vals = {    'maskname':      mask_name,
                        'cameras':       cameras,
                        'opamps':        opamps,         
                        'datadir':       raw_data_dir,
                        'template':      raw_filename_template,
                        'tags':          ''                      }

In [15]:
from quickreduce_funcs import get_all_filedata

if do_bias:
    data, headers = get_all_filedata(filenum_dict=filenumbers,
                                      cut_bias_cols=cut_bias_cols,
                                      **common_func_vals)
    
    if bias_array_dict is None or 'bias' in data.keys():
        bias_array_dict = data.pop('bias')
        bias_header_dict = headers.pop('bias')
    else:
        print("Can't find the bias data!")

In [16]:
from quickreduce_funcs import debug_plots, scrub_header

if do_bias:
    master_bias_data_dict = {}
    master_bias_header_dict = {}
    for camera,camera_bias_dict in bias_array_dict.items():
        master_camera_bias_data_dict = {}
        master_camera_bias_header_dict = {}
        for opamp,opampdict in camera_bias_dict.items():
            opamparray = np.asarray(list(opampdict.values()))
            if make_debug_plots:
                debug_plots(opamparray,camera, opamp)
            opamparray = np.median(opamparray,axis=0)
            header = list(bias_header_dict[camera][opamp].values())[0]
            header = scrub_header(header,opamparray.shape)
            header.add_history("Median Master Bias done by quickreduce on {}".format(date))
            outhdu = fits.PrimaryHDU(data=opamparray ,header=header)
            outname = master_valadded_fname_template.format(cam=camera, imtype='bias',maskname=mask_name, 
                                                            opamp=opamp,tags=common_func_vals['tags'])
            outhdu.writeto(os.path.join(common_func_vals['datadir'], outname) ,overwrite=True)
            master_camera_bias_data_dict[opamp] = opamparray
            master_camera_bias_header_dict[opamp] = header
        master_bias_data_dict[camera] = master_camera_bias_data_dict
        master_bias_header_dict[camera] = master_camera_bias_header_dict
        print("Completed generation of master bias")
        print("Results saved to {}".format(common_func_vals['datadir']))
        del bias_array_dict, bias_header_dict
# else:
#     ## load in master bias
#     print("Loaded master bias")
#     print("Loaded from {}".format(common_func_vals['datadir']))
    
# master_aux_data['bias'] = master_bias_data_dict
# master_aux_headers['bias'] = master_bias_header_dict

filenumbers.pop('bias')  

array([597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609,
       610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622,
       623, 624, 625, 626])

In [17]:
common_func_vals['datadir']= product_dir
common_func_vals['template'] = valadded_filename_template
common_func_vals['tags'] = '.b'        

In [18]:
if do_bias:
    filenames = {}
    for datatype,filenums in filenumbers.items():
        filenames[datatype] = {}
        datadict = data[datatype]
        headerdict = headers[datatype]
        for camera,camera_data_dict in datadict.items():
            for opamp,opamparrays in camera_data_dict.items():
                master_bias = master_bias_data_dict[camera][opamp]
                for filenum in filenums:
                    opamparray = opamparrays[filenum].astype(float)
                    header = headerdict[camera][opamp][filenum]
                    opamparray -= master_bias.astype(float)
                    header = scrub_header(header,opamparray.shape)
                    header.add_history("Bias Subtracted done by quickreduce on {}".format(date))
                    outhdu = fits.PrimaryHDU(data=opamparray ,header=header)  
                    outname = valadded_filename_template.format(cam=camera,imtype=datatype, maskname=mask_name,
                                                                opamp=opamp,filenum=filenum,tags=common_func_vals['tags'])
                    outhdu.writeto(os.path.join(common_func_vals['datadir'] ,outname) ,overwrite=True)           
                    headerdict[camera][opamp][filenum] = header
                    datadict[camera][opamp][filenum] = opamparray
        print("Completed bias subtraction for {}".format(datatype))
        print("Results saved to {}".format(common_func_vals['datadir']))

In [19]:
import PyCosmic

old_tag = common_func_vals['tags']
new_tag = common_func_vals['tags']+'c'
    
if do_cr:
    if len(data.keys()) == 0:
        data, headers = get_all_filedata( filenum_dict=filenumbers, **common_func_vals )
    for datatype in data.keys():
        for camera in data[datatype].keys():
            for opamp in data[datatype][camera].keys():
                for filenum in filenumbers[datatype]:
                    filename = valadded_filename_template.format(cam=camera, imtype=datatype, maskname=mask_name,
                                                                 opamp=opamp, filenum=filenum, tags=old_tag)
                    filename = os.path.join(common_func_vals['datadir'],filename)
                    savefile = filename.replace(old_tag+'.fits',new_tag+'.fits')
                    maskfile = filename.replace(old_tag+'.fits',old_tag+'.crmask.fits')
                    print("\nFor image type: {}, shoe: {},  opamp: {}, filenum: {}".format(datatype,camera,opamp,filenum))
                    outdat,pycosmask,pyheader = PyCosmic.detCos(filename,maskfile,savefile,rdnoise='ENOISE',sigma_det=8,
                                                                gain='EGAIN',verbose=True,return_data=True)
                    data[datatype][camera][opamp][filenum] = outdat
                    ## Plot the image
                    plt.figure()
                    pycosmask = pycosmask - np.min(pycosmask) + 1e-4
                    plt.imshow(np.log(pycosmask),'gray',origin='lowerleft')
                    plt.savefig(maskfile.replace('.fits','.png'),dpi=1200)
                    plt.close()
                        
common_func_vals['tags'] = new_tag
del old_tag, new_tag

In [20]:
if do_flat:
    if len(data.keys()) == 0:
        data, headers = get_all_filedata( filenum_dict=filenumbers, **common_func_vals )
    if twiflat_array_dict is None or 'twiflat' in data.keys():
        twiflat_array_dict = data.pop('twiflat')
        twiflat_header_dict = headers.pop('twiflat')
    else:
        print("Can't find the twilight flats!")

In [21]:
if do_flat:
    common_func_vals['tags'] = common_func_vals['tags']+'f'
    master_twiflat_data_dict = {}
    master_twiflat_header_dict = {}
    for camera,camera_data_dict in twiflat_array_dict.items():
        master_camera_twiflat_data_dict = {}
        master_camera_twiflat_header_dict = {}
        for opamp,opampdict in camera_data_dict.items():
            opamparrays = np.asarray(list(opampdict.values()))
            heads = twiflat_header_dict[camera][opamp]
            header = list(heads.values())[0]
            if header['SHOE'].lower() != camera:
                print("In twiflat: Camera did not match the header's shoe variable")
            if header['OPAMP'] != opamp:
                print("In twiflat: Opamp did not match the header's opamp variable")
            opamparray = opamparrays.sum(axis=0)  
            header.add_history("Master twiflat summed by quickreduce on {}".format(date))
            header['TOTEXPT'] = np.sum([float(heads[key]['EXPTIME']) for key in heads.keys()])
            header.remove('EXPTIME')
            outhdu = fits.PrimaryHDU(data=opamparray ,header=header)  
            outname = master_valadded_fname_template.format(cam=camera, imtype="twilightflat", 
                                                            maskname=mask_name, opamp=opamp, 
                                                            tags=common_func_vals['tags'])
            outhdu.writeto(os.path.join(common_func_vals['datadir'] ,outname) ,overwrite=True)
            master_camera_twiflat_data_dict[opamp] = opamparray
            master_camera_twiflat_header_dict[opamp] = header
            if make_debug_plots:
                typical_sep = np.mean(opamparrays[0,:,:])- np.mean(opamparrays[-1,:,:])
                if typical_sep < 10:
                    typical_sep = 10
                debug_plots(opamparrays,camera, opamp, filetype='Twilight Flat',typical_sep=typical_sep,np_oper=np.sum)
        master_twiflat_data_dict[camera] = master_camera_twiflat_data_dict
        master_twiflat_header_dict[camera] = master_camera_twiflat_header_dict

    master_aux_data['twiflat'] = master_twiflat_data_dict
    master_aux_headers['twiflat'] = master_twiflat_header_dict

    print("Completed creation of master twiflat")
    print("Results saved to {}".format(common_func_vals['datadir']))
    #del twiflat_array_dict, twiflat_header_dict
# else:
#     ## load in master twilight flat
#     print("Loaded master twilight flat")
#     print("Loaded from {}".format(common_func_vals['datadir']))
    
# master_aux_data['twiflat'] = master_twiflat_data_dict
# master_aux_headers['twiflat'] = master_twiflat_header_dict   
    
filenumbers.pop('twiflat')

Completed creation of master twiflat
Results saved to C:\Users\kremin\OneDrive\Research\M2FSReductions\A02\data_products


array([582, 583, 584, 585, 586, 587, 588, 589, 590, 591])

In [24]:
if do_flat:
    for datatype in data.keys():
        datadict = data[datatype]
        headerdict = headers[datatype]
        for camera,camera_data_dict in datadict.items():
            for opamp,opamparrays in camera_data_dict.items():
                master_twiflat = master_twiflat_data_dict[camera][opamp]
                for filnum,filearray in opamparrays.items():
                    opamparray = opamparrays[filnum].astype(float)
                    header = headerdict[camera][opamp][filnum]
                    opamparray /= master_twiflat.astype(float)
                    header.add_history("Flat correction done by quickreduce on {}".format(date))
                    outhdu = fits.PrimaryHDU(data=opamparray ,header=header)
                    filename = valadded_filename_template.format(cam=camera, imtype=datatype, 
                                                                 maskname=mask_name, opamp=opamp, 
                                                                 filenum=filnum, tags=common_func_vals['tags'])
                    filename = os.path.join(common_func_vals['datadir'],filename)
                    outhdu.writeto(filename ,overwrite=True)           
                    headerdict[camera][opamp][filnum] = header
                    datadict[camera][opamp][filnum] = opamparray

        print("Completed flattening for {}".format(datatype))
        print("Results saved to {}".format(common_func_vals['datadir']))

Completed flattening for thar
Results saved to C:\Users\kremin\OneDrive\Research\M2FSReductions\A02\data_products
Completed flattening for comp
Results saved to C:\Users\kremin\OneDrive\Research\M2FSReductions\A02\data_products
Completed flattening for science
Results saved to C:\Users\kremin\OneDrive\Research\M2FSReductions\A02\data_products
Completed flattening for fibmap
Results saved to C:\Users\kremin\OneDrive\Research\M2FSReductions\A02\data_products


In [25]:
if print_headers:
    print(list(headers['science'][camera][opamp].values())[0].keys)

<bound method Header.iterkeys of SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1024                                                  
NAXIS2  =                 1028                                                  
BUNIT   = 'DU/PIXEL'                                                            
ORIGIN  = 'LCO/OCIW'                                                            
OBSERVER= 'Bailey/Tucker/Spencer/Olsz' / observer name                          
TELESCOP= 'Clay'                       / telescope                              
SITENAME= 'LCO'                                                                 
SITEALT =                 2405         / meters                                 
SITELAT =            -29.01423                                              

In [26]:
#transforms = {4: 'l', 1: 'ul', 2: 'u', 3: 'none'}
#camloc = {4:'br',1:'ur',2:'ul',3:'bl'}

xorients = {-1:'l', 1:'r'}
yorients = {-1:'b', 1:'u'}

In [27]:
del data
data = {}
common_func_vals['datadir'] = product_dir
common_func_vals['template'] = valadded_filename_template

In [28]:
if do_stitch:
    if len(data.keys()) == 0:
        data, headers = get_all_filedata( filenum_dict=filenumbers, **common_func_vals )
    common_func_vals['datadir'] = stitched_dir
    common_func_vals['template'] = stitched_filename_template
    for imtype in data.keys():
        data_dict = data[imtype]
        header_dict = headers[imtype]
        for camera in data_dict.keys():
            cam_data_dict = data_dict[camera]
            cam_header_dict = header_dict[camera]
            data[imtype][camera]['stitched'] = {}
            headers[imtype][camera]['stitched'] = {}
            for filenum in filenumbers[imtype]:
                img = {}
                for opamp in camera_data_dict.keys():
                    header = cam_header_dict[opamp][filenum]
                    xsign = np.sign(header['CHOFFX'])
                    ysign = np.sign(header['CHOFFY'])
                    location = yorients[ysign]+xorients[xsign]
                    print("Imtype: {}  In filenum: {} Camera: {} Opamp: {} located at {}".format(imtype,filenum,camera,opamp,location))
                    img[location] = cam_data_dict[opamp][filenum]

                trans = {}
                ## Transform opamps to the correct directions
                trans['bl'] = img['bl']
                trans['br'] = np.fliplr(img['br'])
                trans['ul'] = np.flipud(img['ul'])
                trans['ur'] = np.fliplr(np.flipud(img['ur']))
                del img

                y_bl,x_bl = trans['bl'].shape
                y_ul,x_ul = trans['ul'].shape
                y_br,x_br = trans['br'].shape
                y_ur,x_ur = trans['ur'].shape

                ## Make sure the shapes work
                if y_bl != y_br or y_ul != y_ur:
                    print("yr and yl not the same")
                if x_bl != x_ul or x_br != x_ur:
                    print("xb and xu not the same")

                ## Create the full-sized image array
                merged = np.ndarray(shape=(y_bl+y_ul,x_bl+x_br))

                ## Assign the opamps to the correct region of the array
                ## bl
                merged[:y_bl,:x_bl] = trans['bl']
                ## ul
                merged[y_bl:,:x_bl] = trans['ul']
                ## br
                merged[:y_bl,x_bl:] = trans['br']
                ## ur
                merged[y_bl:,x_bl:] = trans['ur']

                opamp = 1
                header = cam_header_dict[opamp][filenum].copy()
                header['NAXIS1'] = int(y_bl+y_ul)
                header['NAXIS2'] = int(x_bl+x_br)
                header['FILENAME'] = header['FILENAME'].split('c{}'.format(opamp))[0]
                for key in ['OPAMP','CHOFFX','CHOFFY']:
                    header.remove(key)
                header.add_history("Stitched 4 opamps by quickreduce on {}".format(date))
                outhdu = fits.PrimaryHDU(data=merged ,header=header)
                filename = stitched_filename_template.format(imtype=imtype,cam=camera, filenum=filenum,
                                                             maskname=mask_name, tags=common_func_vals['tags'])
                filename = os.path.join(common_func_vals['datadir'],filename)
                outhdu.writeto(filename ,overwrite=True)

                data[imtype][camera]['stitched'][filenum] = merged
                header[imtype][camera]['stitched'][filenum] = header

                ## Plot the image
                plt.figure()
                merged = merged - np.min(merged) + 1e-4
                plt.imshow(np.log(merged),'gray',origin='lowerleft')
                plt.savefig(filename.replace('.fits','.png'),dpi=1200)
                plt.show()
            for opamp in opamps:
                data[imtype][camera].remove(opamp)
                header[imtype][camera].remove(opamp)

Imtype: thar  In filenum: 627 Camera: r Opamp: 1 located at ur
Imtype: thar  In filenum: 627 Camera: r Opamp: 2 located at ul
Imtype: thar  In filenum: 627 Camera: r Opamp: 3 located at bl
Imtype: thar  In filenum: 627 Camera: r Opamp: 4 located at br


KeyError: "Keyword 'THAR' not found."

In [ ]:
common_func_vals['opamps'] = 'stitched'

In [ ]:
from quickreduce_funcs import get_all_stitcheddata

if do_apcut:
    if len(data.keys()) == 0:
        data, headers = get_all_stitcheddata( filnum_dict=filenumbers, **common_func_vals )
    if fibermap_array_dict is None or 'fibmap' in data.keys():
        fibermap_array_dict = data.pop('fibmap')
        fibermap_header_dict = headers.pop('fibmap')
    else:
        print("Can't find the fiber maps!")

In [ ]:
if do_apcut:
    master_fibm_data_dict = {}
    master_fibm_header_dict = {}

    for camera,camera_data_dict in fibermap_array_dict.items():
        
        opamp = 'stitched'
        
        heads = fibm_header_dict[camera][opamp]
        master_bias = master_bias_data_dict[camera][opamp]
        header = list(heads.values())[0]
        
        if header['SHOE'].lower() != camera:
            print("In fibermap: Camera did not match the header's shoe variable")
        if header['OPAMP'] != opamp:
            print("In fibermap: Opamp did not match the header's opamp variable")
            
        
        opamparray = np.asarray(list(heads.values())).sum(axis=0)
        header.add_history("Master fibermap done by quickreduce on {}".format(date))
        
        outhdu = fits.PrimaryHDU(data=opamparray ,header=header)  
        outname = master_stitched_fname_template.format(cam=camera, imtype="fibermap", 
                                                        maskname=mask_name, 
                                                        tags=common_func_vals['tags'])
        outname = os.path.join(common_func_vals['dirname'],outname)
        outhdu.writeto( outname ,overwrite=True)

        if make_debug_plots:
            typical_sep = np.mean(opamparrays[0,:,:])- np.mean(opamparrays[-1,:,:])
            if typical_sep < 10:
                typical_sep = 10
            debug_plots(opamparrays,camera, opamp, filetype='Fibermap',typical_sep=typical_sep,np_oper=np.sum)
        master_fibm_data_dict[camera] = opamparray
        master_fibm_header_dict[camera] = header

    master_aux_data['fibmap'] = master_fibm_data_dict
    master_aux_headers['fibmap'] = master_fibm_header_dict

    print("Completed master fibermap")
    print("Results saved to {}".format(common_func_vals['datadir']))
    #del fibm_array_dict, fibm_header_dict

# else:
#     ## load in master fibermap
#     print("Loaded master fibermap")
#     print("Loaded from {}".format(common_func_vals['datadir']))
    
# master_aux_data['bias'] = master_fibm_data_dict
# master_aux_headers['bias'] = master_fibm_header_dict       
    
filenumbers.pop('fibmap')

In [ ]:
common_func_vals['datadir'] = stitched_dir
common_func_vals['template'] = stitched_filename_template